In [28]:
# -*- coding: utf-8 -*-
#  Copyright 2025 United Kingdom Research and Innovation
#  Copyright 2025 The University of Manchester
#  Copyright 2025 Technical University of Denmark
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
#
#   Authored by:    Hannah Robarts (UKRI-STFC)
#                   Laura Murgatroyd (UKRI-STFC)

# CIL-ESRF pipeline

This notebook contains example scripts for loading, pre-processing, reconstructing and visualising tomography data collected at ESRF beamlines. The steps are designed to be adaptable for different kinds of synchrotron datasets and covers commonly used methods, including:
- Extracting experiment information and configuring a CIL `AcquisitionData` object
- Applying CIL pre-processors, including `Normaliser`, `TransmissionAbsorptionConverter`, `CentreOfRotationCorrector` and `PaganinProcessor`
- Reconstructing using filtered back projection with CIL's wrapper for tigre

This example uses dataset tomo_00065 from the TomoBank [[1](https://iopscience.iop.org/article/10.1088/1361-6501/aa9c19)] multidistance dataset. The sample is a steel sphere measured at various propagation distances to demonstrate the effect of propagation based phase contrast imaging.

The tomo_00065.h5 dataset can be retrieved from https://tomobank.readthedocs.io/en/latest/source/data/docs.data.phasecontrast.html#multi-distance using:

`wget https://g-a0400.fd635.8443.data.globus.org/tomo_00064_to_00067/tomo_00065.h5`

[1] De Carlo, Francesco, et al. “TomoBank: a tomographic data repository for computational x-ray science.” Measurement Science and Technology 29.3 (2018): 034004. http://www.doi.org/10.1088/1361-6501/aa9c19

##### Load dependencies

In [29]:
# CIL methods
from cil.framework import DataContainer
from cil.utilities.display import show2D, show_geometry
from cil.utilities.jupyter import islicer
from cil.io.utilities import HDF5_utilities
from cil.io import TIFFWriter
from cil.processors import Normaliser, RingRemover, TransmissionAbsorptionConverter, CentreOfRotationCorrector, PaganinProcessor
from cil.recon import FBP
# Additional packages
import numpy as np # conda install numpy
import matplotlib.pyplot as plt # conda install matplotlib
# Custom methods
from esrf_code.HDF5_ParallelDataReader import HDF5_ParallelDataReader


##### Load the data

Choose the file and use `HDF5_utilities` to print the metadata and find the locations of data and scan information within the file. We see there is a lot of information about the experiment we can use to help with the processing and reconstruction

In [ ]:
filename = '/mnt/share/materials/SIRF/Fully3D/CIL/Phase/tomo_00065.h5'
HDF5_utilities.print_metadata(filename) # comment out if you don't want to see the metadata

Configure paths to the relevant data and metadata in the file, then read the data using the generic `HDF5_ParallelDataReader`


In [31]:
reader = HDF5_ParallelDataReader(filename, 
                                 dataset_path=('exchange/data'),
                                 distance_units='mm', angle_units='degree')

reader.configure_angles(angles_path='exchange/theta', HDF5_units='degree')

reader.configure_pixel_sizes('measurement/instrument/detector/x_actual_pixel_size',
                             'measurement/instrument/detector/y_actual_pixel_size',
                             HDF5_units = 'um')

reader.configure_normalisation_data(flatfield_path='exchange/data_white',
                                    darkfield_path='exchange/data_dark')

reader.configure_sample_detector_distance(sample_detector_distance=58, HDF5_units='mm') # required for phase retrieval

data = reader.read()

energy = HDF5_utilities.read(filename, 'measurement/instrument/monochromator/energy') # required for phase retrieval

Use `islicer` to visualise the data. Try looking through the projections by sliding the slice index slider.

In [ ]:
islicer(data)

And `show_geometry()` to check the orientation of the sample and detector

In [ ]:
show_geometry(data.geometry)

##### Normalise

The uneven background suggests the data needs to be normalised. We use the flat and dark scans that we loaded as part of the data reader and the CIL `Normaliser` method. Here we have multiple flat and dark scans so we take the mean along the first axis of each. To learn more about the parameters for `Normaliser`, check CIL's documentation https://tomographicimaging.github.io/CIL/v24.3.0/processors/#data-normaliser.

In [34]:
data_before = data.copy() # make a copy of the data for comparison

In [ ]:
processor = Normaliser(flat_field=np.mean(reader.flatfield.array, axis = 0), dark_field=np.mean(reader.darkfield.array, axis = 0))
processor.set_input(data_before)
data = processor.get_output()

# Use the show2D method to check the effect of the normalisation
show2D([data_before, data],
       title=['Before Normalisation', 'After Normalisation'])

Also look at a vertical slice of the data which will allow us to compare the effect on the sinograms

In [ ]:
show2D([data_before, data], slice_list=('vertical',420),
       title=['Before Normalisation', 'After Normalisation'])

##### Transmission to absorption 

Next we use the CIL `TransmissionAbsorptionConverter` which applies the Beer-Lambert law, to view the data in the absorption domain. If there are negative numbers in the data, specify a low value in `min_intensity` to clip these values before calculating $-log()$, check CIL's documentation for more details about configuring this processor https://tomographicimaging.github.io/CIL/v24.3.0/processors/#transmission-to-absorption-converter

In [37]:
data_before = data.copy()

In [ ]:
data = TransmissionAbsorptionConverter()(data_before)
show2D([data_before, data], ['Before transmission to absorption correction','After transmission to absorption correction'])


##### Filtered back projection

Next we use the CIL Filtered Back Projection `FBP` method to check the reconstruction on a single vertical slice of the data. The FBP method in the recon class uses `tigre` by default but can alternatively be configured for use with the  `backend = astra`. These use projectors from the tigre and astra packages respectively, see CIL's documentation for more details https://tomographicimaging.github.io/CIL/v24.3.0/recon/#fbp-reconstructor-for-parallel-beam-geometry.

In [ ]:
vertical_slice = 460
data_slice = data.get_slice(vertical=vertical_slice)
reco_before = FBP(data_slice).run(verbose=False)
show2D(reco_before)

##### Centre of rotation correction

Various artefacts can be observed in the reconstruction if the sample is not perfectly at the centre of the rotation stage. This dataset is from a parallel beam experiment and it has projections from 360 degrees around the sample, which results in a doubling effect if the centre of rotation is offset. We can remove the artefacts by accounting for the offset in the reconstruction. The CIL `CentreOfRotationCorrector.xcorrelation` processor finds the centre of rotation offset automatically by comparing projections 360 degrees apart and minimising the difference between them. CIL's documentation also contains details of other methods for correcting the centre of rotation https://tomographicimaging.github.io/CIL/v24.3.0/processors/#centre-of-rotation-corrector

In [40]:
data_before = data.copy()

In [ ]:
data = CentreOfRotationCorrector.xcorrelation()(data)

# Check the effect on the reconstruction
data_slice = data.get_slice(vertical=vertical_slice)
reco = FBP(data_slice).run(verbose=False)
show2D([reco_before, reco],
['Before centre of rotation correction','After centre of rotation correction'])

Print the geometry to see the rotation axis has been changed

In [ ]:
print("Centre of rotation before {}, rotation axis position {}"
      .format(data_before.geometry.get_centre_of_rotation(distance_units='pixels'), data_before.geometry.config.system.rotation_axis.position))
print("Centre of rotation after {}, rotation axis position {}"
      .format(data.geometry.get_centre_of_rotation(distance_units='pixels'), data.geometry.config.system.rotation_axis.position))

Alternatively, we can loop through different pixel offsets manually and view the reconstructions using `islicer` to choose the offset where rotation artefacts are minimised. Run the cell and vary the slice index to see the effect of using different pixel offsets.

In [ ]:
array_list = []
pixel_offsets = [-10, -15, -20, -25]
for p in pixel_offsets:
    data_before.geometry.set_centre_of_rotation(p, distance_units='pixels')
    data_slice = data_before.get_slice(vertical=vertical_slice)
    reco_test = FBP(data_slice).run(verbose=False)
    array_list.append(reco_test.array)
DC = DataContainer(np.stack(array_list, axis=0), dimension_labels=tuple(['Centre of rotation offset']) + reco_test.geometry.dimension_labels)
islicer(DC, title=tuple(['Centre of rotation offset: ' + str(p)  + ', index: ' for p in pixel_offsets]))

##### Ring removal

Ring artefacts appear in the reconstruction where dead or varying pixels remain in the projections. Various methods exist to remove these from the reconstruction. Here we use the CIL `RingRemover` which removes stripes in the sinogram via a wavelet decomposition method. Try varying different parameters on a vertical slice of the dataset and see the effect on the rings in the reconstruction (for slice 460 there is a small ring at the centre of the reconstruction)
- The `decNum` parameter defines the number of wavelet decompositions used. Increasing decNum will increase the ring remover strength, but increases the computational effort and may distort the shape of the data.
- `wname` defines the filter name to use from `'db1' -- 'db35', 'haar'` - increasing the wavelet filter number increases the strength of the ring removal, but also increases the computational effort
- `sigma` describes the damping parameter in Fourier space - increasing sigma, increases the size of artefacts which can be removed

Find more details about the ring remover method here https://tomographicimaging.github.io/CIL/v24.3.0/processors/#ring-remover

In [44]:
data_before = data.copy()
reco_before = reco.copy()

In [ ]:
array_list = []
array_list.append(reco_before.array) # include the original reconstruction for comparison
sigma = 0.01
wname = "db5"
decNum_list = [1, 2, 3, 4]
for d in decNum_list:
    data_slice = data.get_slice(vertical=vertical_slice)
    data_slice = RingRemover(decNum = d, wname = wname, sigma = sigma,  info = False)(data_slice)
    reco_test = FBP(data_slice).run(verbose=False)
    array_list.append(reco_test.array)
DC = DataContainer(np.stack(array_list, axis=0), dimension_labels=tuple(['Ring remover decNum']) + reco.geometry.dimension_labels)
islicer(DC, title=tuple(['No ring remover'] + ['Ring remover decNum: ' + str(p) + ', index: ' for p in decNum_list]))

In [46]:
title = tuple(['No ring remover'] + ['Ring remover decNum: ' + str(p) + ', index: ' for p in decNum_list])


We conclude that using small parameters (e.g. `sigma=0.01, wname="db5"` and `decNum=1`) gives the most effective the ring removal without introducing new artefacts, so we apply this method to the whole dataset

In [ ]:
sigma = 0.01
wname = "db5"
decNum = 1
data = RingRemover(decNum = decNum, wname = wname, sigma = sigma,  info = False)(data_before)

# Compare a slice of the reconstruction
data_slice = data.get_slice(vertical=vertical_slice)
reco = FBP(data_slice).run(verbose=False)
show2D([reco_before, reco],
       title=["Before ring removal", "After ring removal"])

##### Phase retrieval

The bright edges in the reconstruction are an example of edge enhancement due to phase contrast. In this experiment, propagation-based phase contrast imaging was used to exploit the different contrast provided by absorption and phase. Phase retrieval methods can be used to separate out the phase and intensity information. CIL implements the common Paganin phase retrieval method (see [https://doi.org/10.1046/j.1365-2818.2002.01010.x](https://onlinelibrary.wiley.com/doi/10.1046/j.1365-2818.2002.01010.x)) which results in a boost to the signal to noise ratio (SNR) without losing spatial resolution and so is a commonly used pre-processing step.

Run the CIL `PaganinProcessor`
- `delta` and `beta` are the real and complex part of the material refractive index. Increasing the ratio of `delta/beta` increases the strength of the filter, here we've chosen the parameters to remove fringes. Try varying the strength to see the effect on the reconstruction.
- `full_retrieval = False` means the calculation does not include $-log()$. If we apply the phase retrieval before converting to absorption we should use `full_retrieval = True`

For more information about using the `PaganinProcessor` in CIL, check the documentation https://tomographicimaging.github.io/CIL/v24.3.0/processors/#paganin-processor or for a more detailed explanation of the effect of different parameters, see the phase retrieval demo in the deep dive folder [demos/4_Deep_Dives/02_phase_retrieval.ipynb](https://github.com/TomographicImaging/CIL-Demos/blob/main/demos/4_Deep_Dives/02_phase_retrieval.ipynb).

In [48]:
data_before = data.copy()
reco_before = reco.copy()

In [ ]:
delta = 3e-5
beta = 1e-9
processor = PaganinProcessor(delta=delta, beta=beta, 
                             energy=energy, energy_units='keV', 
                             full_retrieval=False)
processor.set_input(data_before)
data = processor.get_output()

# Compare a zoomed-in slice of the reconstruction
data_slice = data.get_slice(vertical=vertical_slice)
reco = FBP(data_slice).run(verbose=False)
show2D([reco_before.array[200:300, 200:300], reco.array[200:300, 200:300]],
       title=["Before phase retrieval", "After phase retrieval"])

We can see that the edge enhancement is reduced and some sample features are more blurred. Plot a cross-section through the edge of the sample to look more closely at the fringes caused by the phase contrast. We can see a sharp peak/ fringe at the sample edge in the reconstruction before the phase retrieval and after phase retrieval the fringe is reduced and the reconstruction SNR is improved.

In [ ]:
plt.plot(reco_before.array[250,200:300])
plt.plot(reco.array[250,200:300])
plt.grid()
plt.xlabel('Horizontal x (pixels)')
plt.ylabel('Intensity')
plt.legend(['Before phase retrieval','After phase retrieval'])

##### The final reconstruction

Reconstruct the whole dataset then view the reconstruction in `islicer`. 

Notice how some of the processor parameters we configured for the single slice might need to be edited when applied to the full dataset, such as the ring remover strength. Similarly, the phase retrieval step works best for vertical slices with neighbouring slices that contain similar materials, therefore we notice artefacts in the first and last vertical slice.

In [52]:
reco = FBP(data).run(verbose=False)

In [ ]:
islicer(reco)

##### Save the processed data

Once we're happy with the reconstruction save the processed data as TIFF files

In [27]:
writer = TIFFWriter()
writer.set_up(data = data, file_name='path_to_data/data.tiff')
# writer.write() # uncomment to save the reconstruction